In [1]:
from ARC import ARC

train, test = ARC.load(303)

train[0]

`setup_ipynb.py` executed, (multi-output enabled for subsequent cell executions).


[
🟢 🔴 🟤
🌑 🟢 🔴
🟤 🌑 🟢
, 
🟢 🔴 🟤 ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
🌑 🟢 🔴 ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
🟤 🌑 🟢 ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ 🟢 🔴 🟤 ⚫ ⚫ ⚫
⚫ ⚫ ⚫ 🌑 🟢 🔴 ⚫ ⚫ ⚫
⚫ ⚫ ⚫ 🟤 🌑 🟢 ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ 🟢 🔴 🟤
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ 🌑 🟢 🔴
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ 🟤 🌑 🟢
]

In [2]:
train.overview()

X:  3x3   🔳  9px ⛓1  🔗1 ,  🟢  3px ⛓3  🔗1 ,  🔴  2px ⛓2  🔗1 ,  🟤  2px ⛓2  🔗2 ,  🌑  2px ⛓2  🔗1 ,                  
Y:  9x9   🔳 27px ⛓3  🔗1 ,  🟢  9px ⛓9  🔗1 ,  🔴  6px ⛓6  🔗3 ,  🟤  6px ⛓6  🔗6 ,  🌑  6px ⛓6  🔗3 ,  ⚫ 54px ⛓2  🔗1 

X:  3x3   🔳  9px ⛓1  🔗1 ,  🔴  6px ⛓2  🔗2 ,  🟠  1px ⛓1  🔗1 ,  🟢  2px ⛓1  🔗1 ,                  
Y:  9x9   🔳 54px ⛓2  🔗2 ,  🔴 36px ⛓5  🔗5 ,  🟠  6px ⛓6  🔗6 ,  🟢 12px ⛓6  🔗6 ,  ⚫ 27px ⛓1  🔗1 

X:  3x3   🔳  9px ⛓1  🔗1 ,  🟠  2px ⛓2  🔗1 ,  🌑  4px ⛓2  🔗2 ,  🟡  1px ⛓1  🔗1 ,  🔴  2px ⛓1  🔗1 ,                  
Y:  9x9   🔳 36px ⛓2  🔗2 ,  🟠  8px ⛓8  🔗4 ,  🌑 16px ⛓8  🔗8 ,  🟡  4px ⛓4  🔗4 ,  🔴  8px ⛓4  🔗4 ,  ⚫ 45px ⛓1  🔗1 



In [3]:
# 3x3 -> 9x9
# For pair k, Y.🌹 = [3, 6, 4][k] * X.🌹
# 3, 6, 4 is also the highest count for each Y0, Y1, Y2.

# let's check the pixel ratios

for X, Y in train:
    for glyph in X.glyphs:
        Σ = lambda G: G[glyph].Σ
        print(glyph,  Σ(X), Σ(Y) / Σ(X))
    print()

🟢 3 3.0
🔴 2 3.0
🟤 2 3.0
🌑 2 3.0

🔴 6 6.0
🟠 1 6.0
🟢 2 6.0

🟠 2 4.0
🌑 4 4.0
🟡 1 4.0
🔴 2 4.0



In [4]:
# yup.

# can we find X in Y?

X, Y = train[0]

X, Y.locate(X)

(
 🟢 🔴 🟤
 🌑 🟢 🔴
 🟤 🌑 🟢,
 
 ▫ ▪ ▪ ▪ ▪ ▪ ▪
 ▪ ▪ ▪ ▪ ▪ ▪ ▪
 ▪ ▪ ▪ ▪ ▪ ▪ ▪
 ▪ ▪ ▪ ▫ ▪ ▪ ▪
 ▪ ▪ ▪ ▪ ▪ ▪ ▪
 ▪ ▪ ▪ ▪ ▪ ▪ ▪
 ▪ ▪ ▪ ▪ ▪ ▪ ▫)

In [5]:
# Yes!
# Looks like Y is a 3x3 of 3x3s.
# And supercells in Y are either empty or X.

# Check this

from Supergrid import Supergrid

for X, Y in train:
    S = Supergrid(Y, sy=3, sx=3)
    S == X


▫ ▪ ▪
▪ ▫ ▪
▪ ▪ ▫


▫ ▪ ▫
▪ ▪ ▫
▫ ▫ ▫


▪ ▫ ▫
▪ ▪ ▪
▫ ▫ ▪

In [6]:
# So we can simplify the puzzle to the binary pattern of filled and empty supercells

from Pair import Pair
from Pairs import Pairs

def fix(pair):
    X, Y = pair
    S = Supergrid(Y, sy=3, sx=3)
    return Pair(X, (S == X).int)

train_ = Pairs(fix @ train)

train_.overview()

X:  3x3   🔳  9px ⛓1  🔗1 ,  🟢  3px ⛓3  🔗1 ,  🔴  2px ⛓2  🔗1 ,  🟤  2px ⛓2  🔗2 ,  🌑  2px ⛓2  🔗1 ,                  ,                  
Y:  3x3   🔳  3px ⛓3  🔗1 ,                  ,                  ,                  ,                  ,  ⚪  3px ⛓3  🔗1 ,  ⚫  6px ⛓2  🔗1 
🟢 🔴 🟤 🌑 removed, ⚪ ⚫ created, 🔳 reduced

X:  3x3   🔳  9px ⛓1  🔗1 ,  🔴  6px ⛓2  🔗2 ,  🟠  1px ⛓1  🔗1 ,  🟢  2px ⛓1  🔗1 ,                  ,                  
Y:  3x3   🔳  6px ⛓2  🔗2 ,                  ,                  ,                  ,  ⚪  6px ⛓2  🔗2 ,  ⚫  3px ⛓1  🔗1 
🔴 🟠 🟢 removed, ⚪ ⚫ created, 🔳 reduced

X:  3x3   🔳  9px ⛓1  🔗1 ,  🟠  2px ⛓2  🔗1 ,  🌑  4px ⛓2  🔗2 ,  🟡  1px ⛓1  🔗1 ,  🔴  2px ⛓1  🔗1 ,                  ,                  
Y:  3x3   🔳  4px ⛓2  🔗2 ,                  ,                  ,                  ,                  ,  ⚫  5px ⛓1  🔗1 ,  ⚪  4px ⛓2  🔗2 
🟠 🌑 🟡 🔴 removed, ⚫ ⚪ created, 🔳 reduced



In [7]:
# Stats for Y match those of the most common glyph in X

# Are the locations the same?

[X[X.most_common()[0]] @ Y.bool for X, Y in train_]

[True, True, True]

In [8]:
# Great, we've solved it!

# Identify most-frequent colour C, consider Y as 3x3 supergrid of 3x3 supercells
# Then whenever X[p] == C, supergrid[p] is X

from Grid import Grid

def solve(X):
    Y = Grid('9x9.⚫')
    S = Supergrid(Y, sy=3, sx=3)
    S[X == X.most_common()[0]] = X
    return Y

[Y @ solve(X)  for (X, Y) in train]


[True, True, True]